---
title: "PSTAT 5A: Sampling and Confidence Intervals"
subtitle: "Lecture 11 - Making Sense of Uncertainty"
author: "Narjes Mathlouthi"
date: 07/22/2025
format:
  revealjs:
    slide-level: 2
    logo: /img/logo.png
    theme: default
    css: /files/lecture_notes/lecture10/new-style.css
    slide-number: true
    chalkboard: true
    preview-links: auto
    footer: "Understanding Data – Sampling and Confidence Intervals © 2025"
jupyter: python3
execute:
  echo: false
  warning: false
  message: false
---


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import scipy.stats as stats
from scipy.stats import norm, t
import warnings
warnings.filterwarnings('ignore')

# Set style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")

# Custom color palette
colors = {
    'primary': '#3b82f6',
    'secondary': '#f59e0b', 
    'success': '#10b981',
    'danger': '#ef4444',
    'info': '#8b5cf6',
    'light': '#f8fafc',
    'dark': '#1f2937'
}

```{=html}
<a href="https://pstat5a.com/schedule.html" class="main-page-btn">🏠 Back to Main Page</a>
```

# Welcome to Lecture 11 {.center}

**From Samples to Populations: Understanding Uncertainty**

<!-- <i>**"In statistics, we make educated guesses about the whole by looking at a part"** </i> -->

---

## 📢 Important Announcements

:::: {.columns}
::: {.column width="50%"}
### 📝 Quiz 2 Details
**When:**  
- 📅 **Date:** Friday, July 25  
- ⏰ **Window:** 7 AM – 12 AM  
- ⏳ **Duration:** 1 hour once started

**Where:** 💻 Online via Canvas

**Covers:** Material from Weeks 3-4
:::

::: {.column width="50%"}
### 📚 What to Expect
- Discrete & continuous distributions
- Probability calculations
- Expected value & variance
- Normal distribution applications
- **Note:** Upload photos of written work for calculation problems
:::
::::

## What We'll Learn Today 🎯 

:::: {.columns}
::: {.column}
**Big Ideas:**

- How **sample means** behave (they're surprisingly predictable!)
- Why we use **intervals** instead of *single* numbers
- How confident we can be in our estimates
- Planning studies for the right precision
:::
::: {.column}
**Skills You'll Gain:**

- Build **confidence intervals** step-by-step
- Interpret what confidence really means
- Choose appropriate **sample sizes**
- Avoid common mistakes in interpretation
:::
::::

---

## From Observation ➡️ Experimentation: Why Design Matters {.smaller}

:::: {.incremental}
- **Observational Study:** Passively record what already happens — good for spotting patterns, but hidden *confounders* can fool us.
- **Controlled Experiment:** Actively assign treatments; **randomization** balances the lurking variables so we can talk about *cause*.
- **Randomization & Replication:** Twin shields that protect us from bias and one‑off flukes.
- **Probability Framework:** Sample space, events, and probabilities let us *quantify* uncertainty.
::::

In [ ]:
# Create experiment design visualization
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=("Observational Pattern", "Controlled Experiment (Randomized)",
                   "Anatomy of a Random Experiment", "Impact of Randomization"),
    specs=[[{"type": "scatter"}, {"type": "scatter"}],
           [{"type": "table"}, {"type": "scatter"}]],
    vertical_spacing=0.3,
    horizontal_spacing=0.08
)

# Observational Study
np.random.seed(42)
study_groups = ['Low Exercise', 'Medium Exercise', 'High Exercise']
health_scores = [65, 75, 85]

for i, (group, score) in enumerate(zip(study_groups, health_scores)):
    data = np.random.normal(score, 7, 25)
    fig.add_trace(
        go.Scatter(
            x=[i] * 25 + np.random.normal(0, 0.08, 25),
            y=data,
            mode='markers',
            marker=dict(size=6, color=colors['primary'], opacity=0.6),
            showlegend=False
        ),
        row=1, col=1
    )

# Controlled Experiment
treatment_groups = ['Control', 'Treatment A', 'Treatment B']
treatment_effects = [50, 65, 72]
for i, (group, effect) in enumerate(zip(treatment_groups, treatment_effects)):
    data = np.random.normal(effect, 5, 20)
    fig.add_trace(
        go.Scatter(
            x=[i] * 20 + np.random.normal(0, 0.08, 20),
            y=data,
            mode='markers',
            marker=dict(size=6, color=colors['success'], opacity=0.7),
            showlegend=False
        ),
        row=1, col=2
    )

# Fixed Table - Random Experiment Components
fig.add_trace(
    go.Table(
        header=dict(
            values=["Component", "Definition", "Example"],
            fill_color='#f8fafc',
            font=dict(size=10, color='#1e293b'),
            align="left"
        ),
        cells=dict(
            values=[
                ["Sample Space (S)", "Event (E)", "Probability P(E)", "Randomization", "Replication"],
                ["All possible outcomes", "Subset of sample space", "Likelihood of event", "Random assignment", "Repeated trials"],
                ["Coin: {H, T}", "Getting heads: {H}", "P(Heads) = 0.5", "Random treatment assignment", "Flip coin 100 times"]
            ],
            fill_color='white',
            font=dict(size=9),
            align="left",
            height=22
        )
    ),
    row=2, col=1
)

# Better visualization: Randomization effect demonstration
np.random.seed(123)
# Without randomization - biased groups
without_rand_control = np.random.normal(45, 8, 30)  # Sicker patients
without_rand_treatment = np.random.normal(75, 6, 30)  # Healthier patients

# With randomization - balanced groups  
with_rand_control = np.random.normal(60, 7, 30)
with_rand_treatment = np.random.normal(70, 7, 30)

# Plot without randomization
fig.add_trace(
    go.Scatter(
        x=[-0.2] * 30 + np.random.normal(0, 0.05, 30),
        y=without_rand_control,
        mode='markers',
        marker=dict(size=5, color=colors['danger'], opacity=0.6),
        name='Without Randomization',
        showlegend=False
    ),
    row=2, col=2
)

fig.add_trace(
    go.Scatter(
        x=[0.2] * 30 + np.random.normal(0, 0.05, 30),
        y=without_rand_treatment,
        mode='markers',
        marker=dict(size=5, color=colors['danger'], opacity=0.6),
        showlegend=False
    ),
    row=2, col=2
)

# Plot with randomization
fig.add_trace(
    go.Scatter(
        x=[0.8] * 30 + np.random.normal(0, 0.05, 30),
        y=with_rand_control,
        mode='markers',
        marker=dict(size=5, color=colors['success'], opacity=0.6),
        showlegend=False
    ),
    row=2, col=2
)

fig.add_trace(
    go.Scatter(
        x=[1.2] * 30 + np.random.normal(0, 0.05, 30),
        y=with_rand_treatment,
        mode='markers',
        marker=dict(size=5, color=colors['success'], opacity=0.6),
        showlegend=False
    ),
    row=2, col=2
)

# Update layout
fig.update_layout(
    height=500,
    showlegend=False,
    margin=dict(t=40, b=40, l=40, r=40)
)

# Update axes
fig.update_xaxes(title_text="Exercise Level", tickvals=[0,1,2], 
                ticktext=['Low', 'Medium', 'High'], row=1, col=1)
fig.update_yaxes(title_text="Health Score", row=1, col=1)

fig.update_xaxes(title_text="Treatment Group", tickvals=[0,1,2], 
                ticktext=['Control', 'Treatment A', 'Treatment B'], row=1, col=2)
fig.update_yaxes(title_text="Outcome", row=1, col=2)

fig.update_xaxes(title_text="Randomization Approach", 
                tickvals=[0, 1], 
                ticktext=['Without<br>Randomization', 'With<br>Randomization'], 
                row=2, col=2)
fig.update_yaxes(title_text="Health Outcome", row=2, col=2)

fig.show()

## The Big Picture: From Sample to Population 

:::: {.columns }
::: {.column}

Think of this like **trying to understand a huge library by reading just a few books**

In [ ]:
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Example color palette
#colors = {'primary': '#636EFA', 'danger': '#EF553B'}

# Create figure with two subplots
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=("Population (What we want to know)", "Sample (What we actually see)"),
    specs=[[{"type": "scatter"}, {"type": "scatter"}]]
)

# Population - large scattered points
np.random.seed(42)
pop_x = np.random.normal(50, 15, 1000)
pop_y = np.random.normal(50, 15, 1000)

fig.add_trace(
    go.Scatter(
        x=pop_x, y=pop_y,
        mode='markers',
        marker=dict(size=3, color=colors['primary'], opacity=0.6),
        showlegend=False
    ),
    row=1, col=1
)

# Sample - highlighted subset
sample_indices = np.random.choice(1000, 50, replace=False)
sample_x = pop_x[sample_indices]
sample_y = pop_y[sample_indices]

fig.add_trace(
    go.Scatter(
        x=sample_x, y=sample_y,
        mode='markers',
        marker=dict(size=5, color=colors['danger'], opacity=0.9),
        showlegend=False
    ),
    row=1, col=2
)

# Add mean lines with styled annotations
fig.add_hline(
    y=50, line_dash="dash", line_color=colors['primary'], 
    annotation_text="Population Mean (μ) = 50",
    annotation_font_size=14,
    annotation_font_family="Arial Black",
    row=1, col=1
)
fig.add_hline(
    y=np.mean(sample_x), line_dash="dash", line_color=colors['danger'], 
    annotation_text=f"Sample Mean (x̄) = {np.mean(sample_x):.1f}",
    annotation_font_size=14,
    annotation_font_family="Arial Black",
    row=1, col=2
)

# Bold and enlarge subplot titles
for annotation in fig.layout.annotations:
    annotation.font.size = 12
    annotation.font.family = "Arial Black"

# Update overall layout for fit and font
fig.update_layout(
    font=dict(size=12, family="Arial Black"),
    margin=dict(l=30, r=30, t=30, b=30),
    width=900,
    height=500,
    autosize=True
)

fig.show()

:::
::: {.column}
**Key Terms Made Simple:**

- **Population ($\mu$)**: Everyone we care about (like all students at UCSB)
- **Sample ($\bar x$)**: The people we actually measured (like 100 students we surveyed)
- **Parameter**: The true answer we want (population mean)
- **Statistic**: Our best guess (sample mean)
:::
::::

---

## Why Point Estimates Aren't Enough 

:::: {.columns}
::: {.column }
**Imagine asking:** "What's the average height of UCSB students?"

In [ ]:
# Simulate multiple samples
np.random.seed(123)
true_mean = 68  # inches
true_std = 4

sample_means = []
for i in range(20):
    sample = np.random.normal(true_mean, true_std, 30)
    sample_means.append(np.mean(sample))

fig = go.Figure()

# Add true population mean
fig.add_hline(y=true_mean, line_dash="solid", line_color=colors['success'], 
              line_width=3, annotation_text="True Population Mean = 68 inches")

# Add sample means
fig.add_trace(
    go.Scatter(
        x=list(range(1, 21)),
        y=sample_means,
        mode='markers+lines',
        marker=dict(size=10, color=colors['danger']),
        line=dict(color=colors['danger'], width=2),
        name='Sample Means'
    )
)

fig.update_layout(
    title="20 Different Samples Give 20 Different Answers!",
    xaxis_title="Sample Number",
    yaxis_title="Sample Mean Height (inches)",
    height=500,
    width = 900,
    showlegend=False
)

fig.show()

:::
::: {.column }
**The Problem:** Each sample gives a slightly different answer! 

:::{.fragment}
**The Solution:** Use **confidence intervals** to show the range of reasonable values.
:::
:::
::::

---

## Sampling Distributions 

:::: {.columns}
::: {.column }

Think of sampling distributions like 
**"What if we repeated our study 1000 times?"**

In [ ]:
# Create sampling distribution demonstration
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=(
        "Population Distribution", "One Sample (n=30)",
        "Many Sample Means", "Sampling Distribution of x̄"
    ),
    specs=[[{"type": "histogram"}, {"type": "histogram"}],
           [{"type": "histogram"}, {"type": "histogram"}]]
)

# Population
np.random.seed(42)
population = np.random.exponential(2, 10000)  # Skewed population

fig.add_trace(
    go.Histogram(x=population, nbinsx=50, name="Population", 
                marker_color=colors['primary'], opacity=0.7),
    row=1, col=1
)

# One sample
one_sample = np.random.choice(population, 30)
fig.add_trace(
    go.Histogram(x=one_sample, nbinsx=15, name="One Sample", 
                marker_color=colors['danger'], opacity=0.7),
    row=1, col=2
)

# Many samples visualization
sample_means = []
for i in range(1000):
    sample = np.random.choice(population, 30)
    sample_means.append(np.mean(sample))

# Show first 100 sample means
fig.add_trace(
    go.Scatter(
        x=list(range(1, 101)),
        y=sample_means[:100],
        mode='markers',
        marker=dict(size=6, color=colors['secondary']),
        name="First 100 Sample Means"
    ),
    row=2, col=1
)

# Sampling distribution
fig.add_trace(
    go.Histogram(x=sample_means, nbinsx=40, name="Sampling Distribution", 
                marker_color=colors['success'], opacity=0.7),
    row=2, col=2
)

fig.update_layout(height=500, width = 900, showlegend=False)
fig.show()

:::

::: {.column }
**Key Insights:**

1. **Population** can be any shape (skewed, normal, weird)
   
2. **One sample** might not look like the population
   
3. **Sample means** vary from sample to sample
   
4. **Many sample means** form a beautiful normal distribution!
:::
::::

---

## The Central Limit Theorem (CLT) 🎯 

:::: {.columns}
::: {.column }

In [ ]:
#| fig-cap: 'Central Limit Theorem: Larger Samples → More Normal!'

# Interactive CLT demonstration
def create_clt_demo(population_type="uniform", sample_size=30):
    np.random.seed(42)
    
    # Generate different population shapes
    if population_type == "uniform":
        population = np.random.uniform(0, 10, 5000)
        pop_title = "Uniform Population"
    elif population_type == "exponential":
        population = np.random.exponential(2, 5000)
        pop_title = "Right-Skewed Population"
    elif population_type == "bimodal":
        pop1 = np.random.normal(3, 1, 2500)
        pop2 = np.random.normal(7, 1, 2500)
        population = np.concatenate([pop1, pop2])
        pop_title = "Bimodal Population"
    
    # Generate sampling distribution
    sample_means = []
    for i in range(1000):
        sample = np.random.choice(population, sample_size)
        sample_means.append(np.mean(sample))
    
    return population, sample_means, pop_title

# Create demo for different sample sizes
fig = make_subplots(
    rows=2, cols=3,
    subplot_titles=(
        "Population", "n=5 Sample Means", "n=30 Sample Means",
        "Population", "n=10 Sample Means", "n=50 Sample Means"
    )
)

# Uniform population examples
pop_uniform, means_5, _ = create_clt_demo("uniform", 5)
pop_uniform, means_30, _ = create_clt_demo("uniform", 30)

fig.add_trace(go.Histogram(x=pop_uniform, nbinsx=50, name="Population", 
                          marker_color=colors['primary'], opacity=0.7), row=1, col=1)
fig.add_trace(go.Histogram(x=means_5, nbinsx=30, name="n=5", 
                          marker_color=colors['danger'], opacity=0.7), row=1, col=2)
fig.add_trace(go.Histogram(x=means_30, nbinsx=30, name="n=30", 
                          marker_color=colors['success'], opacity=0.7), row=1, col=3)

# Exponential population examples
pop_exp, means_10, _ = create_clt_demo("exponential", 10)
pop_exp, means_50, _ = create_clt_demo("exponential", 50)

fig.add_trace(go.Histogram(x=pop_exp, nbinsx=50, name="Population", 
                          marker_color=colors['primary'], opacity=0.7), row=2, col=1)
fig.add_trace(go.Histogram(x=means_10, nbinsx=30, name="n=10", 
                          marker_color=colors['secondary'], opacity=0.7), row=2, col=2)
fig.add_trace(go.Histogram(x=means_50, nbinsx=30, name="n=50", 
                          marker_color=colors['info'], opacity=0.7), row=2, col=3)

fig.update_layout(height=500, width=900, showlegend=False, 
                 #title="Central Limit Theorem: Larger Samples → More Normal!"
                 )
fig.show()

:::
::: {.column }
**The Magic Rule:** No matter what shape your population is, sample means will be approximately normal!

**The Rule of Thumb:** With **n ≥ 30**, sample means will be approximately normal, regardless of population shape!
:::
::::

---


## Standard Error {.smaller}

:::: {.columns}
::: {.column }
**What it measures**

- **Standard deviation ($\sigma$ or $s$):** spread of *individual* data points  
- **Standard error (SE):** spread of *sample means*

$$
SE = \frac{\sigma}{\sqrt{n}}
\qquad(\text{if } \sigma \text{ is known})
$$

$$
SE = \frac{s}{\sqrt{n}}
\qquad(\text{usual case, } \sigma \text{ unknown})
$$

**Key facts**

- SE shrinks at rate $1/\sqrt{n}$ — every **4×** more observations ⇒ **½** the SE  
- Smaller SE ⇒ narrower confidence intervals and more powerful tests  
<!-- - SE appears in the denominator of every $t$‑statistic you’ll meet -->
:::


::: {.column }

In [ ]:
# Demonstrate effect of sample size on standard error
sample_sizes = np.array([5, 10, 20, 30, 50, 100, 200])
sigma = 10
standard_errors = sigma / np.sqrt(sample_sizes)

fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=sample_sizes,
        y=standard_errors,
        mode='markers+lines',
        marker=dict(size=12, color=colors['primary']),
        line=dict(color=colors['primary'], width=3),
        name='Standard Error'
    )
)

fig.update_layout(
    title="Bigger Samples = Smaller Standard Error",
    xaxis_title="Sample Size (n)",
    yaxis_title="Standard Error",
    height=400,
    showlegend=False
)

fig.add_annotation(
    x=100, y=3,
    text="The bigger the sample,<br>the more precise our estimate!",
    showarrow=True,
    arrowhead=2,
    arrowcolor=colors['danger'],
    font=dict(size=12, color=colors['danger'])
)

fig.show()

:::{.callout-important}
Doubling your sample size doesn't halve the error - you need 4 times the sample for half the error!
:::

:::
::: 


## Confidence Intervals: The Intuitive Idea 

:::: {.columns}
::: {.column }
**Imagine you're trying to guess someone's height just by looking at their shadow...**

In [ ]:
import numpy as np
from scipy.stats import norm
import plotly.graph_objects as go



# Simulation parameters
np.random.seed(42)
true_mean    = 68
sample_mean  = 67.2
se           = 1.5
ci_lower     = sample_mean - 1.96 * se
ci_upper     = sample_mean + 1.96 * se

# X-axis values
x = np.linspace(60, 76, 1000)
# Sampling distribution PDF
y = norm.pdf(x, sample_mean, se)

# Build figure
fig = go.Figure()

# 1) Sampling distribution curve
fig.add_trace(go.Scatter(
    x=x, y=y,
    mode='lines',
    line=dict(color=colors['primary'], width=3),
    name='Sampling Dist.'
))

# 2) Shaded 95% CI region
ci_mask = (x >= ci_lower) & (x <= ci_upper)
fig.add_trace(go.Scatter(
    x=np.concatenate([x[ci_mask], [ci_upper, ci_lower]]),
    y=np.concatenate([y[ci_mask], [0, 0]]),
    fill='toself',
    fillcolor='rgba(59, 130, 246, 0.3)',
    line=dict(color='rgba(0,0,0,0)'),
    name='95% CI'
))

# 3) Vertical lines (no inline annotation_text)
for value, dash, color in [
    (ci_lower,    'dot',    colors['secondary']),
    (sample_mean, 'solid',  colors['danger']),
    (true_mean,   'dash',   colors['success']),
    (ci_upper,    'dot',    colors['secondary'])
]:
    fig.add_vline(x=value, line_dash=dash, line_color=color)

# 4) Annotations above plot for clear, bold labels
annotations = [
    dict(x=ci_lower,    y=1.05, yref='paper',
         text=f"<b>CI Lower = {ci_lower:.1f}</b>",
         showarrow=False, xanchor='center'),
    dict(x=sample_mean, y=1.10, yref='paper',
         text=f"<b>Sample Mean = {sample_mean:.1f}</b>",
         showarrow=False, xanchor='center'),
    dict(x=true_mean,   y=1.05, yref='paper',
         text=f"<b>True Mean = {true_mean}</b>",
         showarrow=False, xanchor='center'),
    dict(x=ci_upper,    y=1.10, yref='paper',
         text=f"<b>CI Upper = {ci_upper:.1f}</b>",
         showarrow=False, xanchor='center'),
]
fig.update_layout(annotations=annotations)

# 5) Layout tweaks
fig.update_layout(
    title=dict(text="95% CI: We're 95% Sure the True Mean is in This Range", x=0.3, xanchor='center'),
    xaxis=dict(title='Height (inches)', range=[60, 76], tick0=60, dtick=2),
    yaxis=dict(title='Probability Density'),
    width=950,
    height=500,
    margin=dict(t=120, b=40, l=40, r=40),
    showlegend=True
)

fig.show()

:::

::: {.column .smaller .fragment}
**Simple Interpretation:** "We're $95\\%$ confident the true average height is between $64.3$ and $70.1$ inches."


:::
::::

## What Exactly *Is* a Confidence Interval? 🤓 {.smaller}

:::: {.columns}
::: {.column .smaller .semi-fade-out}
- A **confidence interval (CI)** is *point estimate* $\pm$ *margin of error*  
$$
    \text{CI} = \text{statistic} \;\pm\; \bigl(\text{critical value}\bigr)\times\bigl(\text{SE}\bigr)
$$

- The “critical value” comes from a probability model (e.g., $z^{\star}$ or $t^{\star}$).  
  
- The **standard error (SE)** captures sampling variation.

**Frequentist meaning**

> If we repeated the study infinitely many times and built a $95 \%$ CI each time, about $95 \%$ of those intervals would cover the true parameter.

(For any one computed interval the parameter is fixed, the *process* has a $95 \%$ success rate, not the individual interval.)

<!-- ```{python}
#| fig-width: 5
#| fig-height: 3
#| echo: false
import numpy as np, plotly.graph_objects as go
np.random.seed(7)
# quick visual: 10 intervals, half miss intentionally
true_mu = 10
means = np.random.normal(true_mu, 2, 10)
ses   = np.repeat(1.2, 10)
ci_l  = means - 1.96*ses
ci_u  = means + 1.96*ses
fig   = go.Figure()
for i,(l,u,m) in enumerate(zip(ci_l,ci_u,means)):
    color = "#10b981" if l<=true_mu<=u else "#ef4444"
    fig.add_trace(go.Scatter(x=[l,u],y=[i,i],mode="lines",line=dict(color=color,width=4)))
    fig.add_trace(go.Scatter(x=[m],y=[i],mode="markers",marker=dict(color=color,size=6)))
fig.add_vline(x=true_mu,line_color="#3b82f6",line_width=3)
fig.update_xaxes(title_text="Value")
fig.update_yaxes(visible=False)
fig.update_layout(height=220,margin=dict(l=20,r=20,t=10,b=10),showlegend=False)
fig.show()
``` -->


:::


::: {.column .smaller .fragment}


:::{.callout-tip}
### What controls the width?

- **Variability** in the data: larger $\sigma$ or $s$ ⇒ wider CI
    
- **Sample size** $n$: width shrinks at rate $1/\sqrt{n}$ 
  
- **Confidence level**: 99 % CIs are wider than 90 % CIs

:::

:::{.callout-important}
### Common pitfalls

- Saying “there is a $95 \%$ probability that $\mu$ lies in this interval” (wrong)  
  
- Interpreting the CI as covering $95 \%$ of *future* observations (it does not)  
  
- Ignoring conditions (normality or CLT) before using the formulae  
:::

In [ ]:
#| echo: false
import numpy as np, plotly.graph_objects as go
np.random.seed(7)
# quick visual: 10 intervals, half miss intentionally
true_mu = 10
means = np.random.normal(true_mu, 2, 10)
ses   = np.repeat(1.2, 10)
ci_l  = means - 1.96*ses
ci_u  = means + 1.96*ses
fig   = go.Figure()
for i,(l,u,m) in enumerate(zip(ci_l,ci_u,means)):
    color = "#10b981" if l<=true_mu<=u else "#ef4444"
    fig.add_trace(go.Scatter(x=[l,u],y=[i,i],mode="lines",line=dict(color=color,width=4)))
    fig.add_trace(go.Scatter(x=[m],y=[i],mode="markers",marker=dict(color=color,size=6)))
fig.add_vline(x=true_mu,line_color="#3b82f6",line_width=3)
fig.update_xaxes(title_text="Value")
fig.update_yaxes(visible=False)
fig.update_layout(height=220,width = 700, margin=dict(l=20,r=20,t=10,b=10),showlegend=False)
fig.show()

:::
::::

---




## Building Confidence Intervals Step-by-Step 

:::: {.columns}
::: {.column }

For Population Means (Most Common Case)

**When we DON'T know the population standard deviation ($\sigma$):**

In [ ]:
#| echo: false
import numpy as np, plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy.stats import t, norm

# ── 0.  Generate one sample  ─────────────────────────────────────────────
np.random.seed(42)
n        = 25
mu_true  = 68          # just for reference (we never use σ)
sigma    = 4
sample   = np.random.normal(mu_true, sigma, n)
xbar     = sample.mean()
s        = sample.std(ddof=1)
se       = s / np.sqrt(n)
df       = n - 1
t_crit   = t.ppf(0.975, df)          # 95 % two-sided
ci_low, ci_high = xbar - t_crit*se, xbar + t_crit*se

# ── 1-4.  Build the four panels  ─────────────────────────────────────────
fig = make_subplots(rows=2, cols=2,
    subplot_titles=(
        "1️⃣  Your Random Sample",
        "2️⃣  Sampling Distribution of $\\bar X$",
        "3️⃣  $t$-Distribution (df = 24)",
        "4️⃣  95 % Confidence Interval"))

# (1) Sample histogram
fig.add_trace(
    go.Histogram(x=sample, nbinsx=10,
                 marker_color="#4285F4", opacity=0.70, showlegend=False),
    row=1, col=1)
fig.add_vline(x=xbar, line_width=3, line_color="black",
              annotation_text=f"$\\bar x$ = {xbar:0.1f}", annotation_position="top left",
              row=1, col=1)

# (2) Sampling distribution (normal curve with ±1 SE band)
x_grid = np.linspace(xbar-4*se, xbar+4*se, 300)
fig.add_trace(
    go.Scatter(x=x_grid, y=norm.pdf(x_grid, xbar, se),
               mode="lines", line_color="#34A853", showlegend=False),
    row=1, col=2)
fig.add_vrect(x0=xbar-se, x1=xbar+se, fillcolor="#34A853", opacity=0.25,
              row=1, col=2)
fig.add_annotation(x=xbar, y=norm.pdf(xbar, xbar, se)*1.1,
                   text="±1 SE", showarrow=False, row=1, col=2)

# (3) t-distribution with shaded 2.5 % tails
x_t = np.linspace(-4,4,400)
y_t = t.pdf(x_t, df)
fig.add_trace(go.Scatter(x=x_t, y=y_t, mode="lines",
                         line_color="#FBBC05", showlegend=False),
              row=2, col=1)
# shade both tails
fig.add_vrect(x0=t_crit,     x1= 4, fillcolor="#FBBC05", opacity=0.25, row=2, col=1)
fig.add_vrect(x0=-4, x1=-t_crit, fillcolor="#FBBC05", opacity=0.25, row=2, col=1)
fig.add_annotation(x=t_crit, y=t.pdf(t_crit, df)*1.05,
                   text=f"$t^*$ = {t_crit:0.2f}", showarrow=False, row=2, col=1)

# (4) Number line + CI
fig.add_trace(go.Scatter(x=[ci_low, ci_high], y=[0,0],
                         mode="markers+lines", line=dict(color="#EA4335", width=6),
                         marker=dict(size=12,color="#EA4335"), showlegend=False),
              row=2, col=2)
fig.add_trace(go.Scatter(x=[xbar], y=[0], mode="markers",
                         marker=dict(size=14, color="black", symbol="x"), showlegend=False),
              row=2, col=2)
fig.update_yaxes(visible=False, row=2, col=2)
fig.add_annotation(x=xbar, y=0.1, text=f"$\\bar x$", showarrow=False, row=2, col=2)
fig.add_annotation(x=ci_low,  y=-0.1, text=f"{ci_low:0.1f}", showarrow=False, row=2, col=2)
fig.add_annotation(x=ci_high, y=-0.1, text=f"{ci_high:0.1f}", showarrow=False, row=2, col=2)

# ──  Cosmetic tweaks  ────────────────────────────────────────────────────
fig.update_layout(height=460, width=820, margin=dict(t=60),
                  title_text="Building a 95 % CI for a Mean (Unknown σ)")
fig.show()

:::
::: {.column }
**The Formula:** $\bar{x} \pm t^* \cdot \frac{s}{\sqrt{n}}$

**Breaking it down:**

- $\bar{x}$ = our sample average (the center of our guess)
  
- $t^*$ = critical value (how many standard errors to go out)
  
- $\frac{s}{\sqrt{n}}$ = standard error (our uncertainty measure)
:::
::::

---

## The t-Distribution: When $\sigma$ is Unknown 

:::: {.columns}
::: {.column }
**Why not use the normal distribution?** Because when we estimate $\sigma$ with $s$, we add extra uncertainty!

In [ ]:
# Compare normal vs t-distributions
x = np.linspace(-4, 4, 1000)

fig = go.Figure()

# Standard normal
y_normal = norm.pdf(x, 0, 1)
fig.add_trace(
    go.Scatter(x=x, y=y_normal, mode='lines', name='Normal (σ known)',
              line=dict(color=colors['primary'], width=3))
)

# t-distributions with different df
for df, color, name in [(5, colors['danger'], 't (df=5)'), 
                       (10, colors['secondary'], 't (df=10)'),
                       (30, colors['success'], 't (df=30)')]:
    y_t = t.pdf(x, df)
    fig.add_trace(
        go.Scatter(x=x, y=y_t, mode='lines', name=name,
                  line=dict(color=color, width=2, dash='dash'))
    )

fig.update_layout(
    title="t-Distribution vs Normal: More Uncertainty with Smaller Samples",
    xaxis_title="Value",
    yaxis_title="Probability Density",
    height=400
)

fig.add_annotation(
    x=0, y=0.3,
    text="As sample size increases,<br>t approaches normal",
    showarrow=True,
    arrowhead=2,
    arrowcolor=colors['info'],
    font=dict(size=12, color=colors['info'])
)

fig.show()

:::
::: {.column }
**Key Points:**

- Small samples ($n < 30$): Use t-distribution
  
- Large samples ($n ≥ 30$): $t$ ≈ normal
  
- Degrees of freedom (df)= $n - 1$
:::
::::

---

## Confidence Intervals for Proportions 

:::: {.columns}
::: {.column}
**For Yes/No questions like:** "What percentage of students prefer online classes?"

In [ ]:
# Proportion confidence interval demonstration
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=("Sample Results", "Confidence Interval")
)

# Sample data
n = 200
x = 120  # successes
p_hat = x / n

# Create sample visualization
categories = ['Prefer Online', 'Prefer In-Person']
counts = [x, n - x]
colors_bar = [colors['success'], colors['danger']]

fig.add_trace(
    go.Bar(x=categories, y=counts, marker_color=colors_bar, name="Sample Counts"),
    row=1, col=1
)

# Confidence interval
se_prop = np.sqrt(p_hat * (1 - p_hat) / n)
z_critical = 1.96  # 95% confidence
margin_error = z_critical * se_prop
ci_lower = p_hat - margin_error
ci_upper = p_hat + margin_error

# CI visualization
fig.add_trace(
    go.Scatter(x=[ci_lower, p_hat, ci_upper], 
               y=[1, 1, 1],
               mode='markers+lines',
               marker=dict(size=[15, 20, 15], color=colors['primary']),
               line=dict(color=colors['primary'], width=5),
               name="95% CI"),
    row=1, col=2
)

fig.update_xaxes(title_text="Response", row=1, col=1)
fig.update_xaxes(title_text="Proportion", range=[0.5, 0.7], row=1, col=2)
fig.update_yaxes(title_text="Count", row=1, col=1)
fig.update_yaxes(title_text="", showticklabels=False, row=1, col=2)

fig.update_layout(height=400, showlegend=False)
fig.show()

:::
::: {.column}
**The Formula:** $\hat{p} \pm z^* \sqrt{\frac{\hat{p}(1-\hat{p})}{n}}$

**Results:** Sample: 60% prefer online (120/200)  
**95% CI:** (53.2%, 66.8%) - We're 95% confident the true percentage is in this range.
:::
::::

---

## What Does "95% Confident" Really Mean? 🤔 {.smaller}

:::: {.columns}
::: {.column .smaller}
**The Biggest Misconception:** "There's a 95% chance the true mean is in our interval"

**Actually:** "If we repeated this study 100 times, about 95 of our intervals would contain the true mean"

In [ ]:
# Simulate 100 confidence intervals
np.random.seed(42)
true_mean = 50
true_std = 10
n = 25
num_intervals = 50

intervals = []
captures = []

for i in range(num_intervals):
    # Generate sample
    sample = np.random.normal(true_mean, true_std, n)
    x_bar = np.mean(sample)
    s = np.std(sample, ddof=1)
    se = s / np.sqrt(n)
    
    # 95% CI
    t_crit = t.ppf(0.975, n-1)
    margin = t_crit * se
    ci_lower = x_bar - margin
    ci_upper = x_bar + margin
    
    intervals.append((ci_lower, ci_upper, x_bar))
    captures.append(ci_lower <= true_mean <= ci_upper)

# Create visualization
fig = go.Figure()

# Add intervals
for i, ((lower, upper, mean), capture) in enumerate(zip(intervals, captures)):
    color = colors['success'] if capture else colors['danger']
    # Confidence interval line
    fig.add_trace(
        go.Scatter(x=[lower, upper], y=[i, i], mode='lines',
                  line=dict(color=color, width=3),
                  showlegend=False)
    )
    # Sample mean point
    fig.add_trace(
        go.Scatter(x=[mean], y=[i], mode='markers',
                  marker=dict(size=6, color=color),
                  showlegend=False)
    )

# Add true mean line
fig.add_vline(x=true_mean, line_dash="solid", line_color=colors['primary'], 
              line_width=4, annotation_text=f"True Mean = {true_mean}")

# Count captures
capture_rate = np.mean(captures) * 100

fig.update_layout(
    title=f"50 Different 95% Confidence Intervals: {capture_rate:.0f}% Capture the True Mean",
    xaxis_title="Value",
    yaxis_title="Study Number",
    height=600
)

fig.add_annotation(
    x=45, y=40,
    text=f"Green = Contains true mean<br>Red = Misses true mean<br>Rate = {capture_rate:.0f}%",
    showarrow=False,
    bgcolor="white",
    bordercolor="black",
    borderwidth=1,
    font=dict(size=12)
)

fig.show()

:::
::: {.column}
**Remember:** The interval either contains the true value or it doesn't - there's no probability involved for a single interval!
:::
::::

---

## Sample Size Planning: Getting the Precision You Want {.smaller}

:::: {.columns}
::: {.column width = "60%"}
**The Question:** "How many people do we need to survey?"

In [ ]:
# Sample size vs margin of error
sigma = 10  # population standard deviation
confidence_levels = [90, 95, 99]
z_values = [1.645, 1.96, 2.576]

fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=("Sample Size vs Margin of Error", "Required Sample Size")
)

# Plot 1: Sample size vs margin of error for different confidence levels
sample_sizes = np.arange(10, 201, 10)

for conf, z_val, color in zip(confidence_levels, z_values, [colors['success'], colors['primary'], colors['danger']]):
    margins = z_val * sigma / np.sqrt(sample_sizes)
    fig.add_trace(
        go.Scatter(x=sample_sizes, y=margins, mode='lines+markers',
                  name=f'{conf}% Confidence',
                  line=dict(color=color, width=3)),
        row=1, col=1
    )

# Plot 2: Required sample size for different margins
desired_margins = np.arange(0.5, 5.1, 0.1)
for conf, z_val, color in zip(confidence_levels, z_values, [colors['success'], colors['primary'], colors['danger']]):
    required_n = (z_val * sigma / desired_margins) ** 2
    fig.add_trace(
        go.Scatter(x=desired_margins, y=required_n, mode='lines',
                  name=f'{conf}% Confidence',
                  line=dict(color=color, width=3),
                  showlegend=False),
        row=1, col=2
    )

fig.update_xaxes(title_text="Sample Size", row=1, col=1)
fig.update_xaxes(title_text="Desired Margin of Error", row=1, col=2)
fig.update_yaxes(title_text="Margin of Error", row=1, col=1)
fig.update_yaxes(title_text="Required Sample Size", row=1, col=2)

fig.update_layout(height=500
                #  title="Planning Your Study: The Trade-offs"
                )
fig.show()

:::
::: {.column .right width = "40%"}
**Key Formula for Means:** $n = \left(\frac{z^* \sigma}{ME}\right)^2$

**Trade-offs:**

- Want smaller margin of error? Need bigger sample
  
- Want higher confidence? Need bigger sample 
   
- Want to save money? Accept wider intervals
:::
::::

---

## Real Example: Student Sleep Study 😴 {.smaller}

:::: {.columns}
::: {.column width ="60%"}
**Research Question:** How many hours do UCSB students sleep per night?

In [ ]:
# Simulate sleep study data
np.random.seed(123)
n = 50
sleep_data = np.random.normal(7.2, 1.5, n)  # Mean 7.2 hours, SD 1.5

# Calculate statistics
x_bar = np.mean(sleep_data)
s = np.std(sleep_data, ddof=1)
se = s / np.sqrt(n)

# 95% confidence interval
df = n - 1
t_crit = t.ppf(0.975, df)
margin = t_crit * se
ci_lower = x_bar - margin
ci_upper = x_bar + margin

# Create comprehensive visualization
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=("Sample Data Distribution", "Step-by-Step Calculation",
                   "Confidence Interval Visualization", "Interpretation"),
    specs=[[{"type": "histogram"}, {"type": "table"}],
           [{"type": "scatter"}, {"type": "scatter"}]]
)

# Sample distribution
fig.add_trace(
    go.Histogram(x=sleep_data, nbinsx=15, name="Sleep Hours",
                marker_color=colors['primary'], opacity=0.7),
    row=1, col=1
)

# Calculation table
calc_data = [
    ["Sample Size (n)", f"{n}"],
    ["Sample Mean (x̄)", f"{x_bar:.2f} hours"],
    ["Sample SD (s)", f"{s:.2f} hours"],
    ["Standard Error", f"{se:.3f} hours"],
    ["t* (95%, df={})".format(df), f"{t_crit:.3f}"],
    ["Margin of Error", f"{margin:.3f} hours"],
    ["95% CI Lower", f"{ci_lower:.2f} hours"],
    ["95% CI Upper", f"{ci_upper:.2f} hours"]
]

fig.add_trace(
    go.Table(
        header=dict(values=["Statistic", "Value"],
                   fill_color=colors['light'],
                   font=dict(size=12)),
        cells=dict(values=list(zip(*calc_data)),
                  fill_color='white',
                  font=dict(size=11))
    ),
    row=1, col=2
)

# CI visualization
fig.add_trace(
    go.Scatter(x=[ci_lower, x_bar, ci_upper], 
               y=[1, 1, 1],
               mode='markers+lines',
               marker=dict(size=[20, 25, 20], 
                          color=[colors['danger'], colors['success'], colors['danger']]),
               line=dict(color=colors['primary'], width=8),
               name="95% Confidence Interval"),
    row=2, col=1
)

# Add text annotations
fig.add_annotation(x=ci_lower, y=1.1, text=f"{ci_lower:.2f}", 
                  showarrow=False, row=2, col=1)
fig.add_annotation(x=x_bar, y=1.1, text=f"{x_bar:.2f}", 
                  showarrow=False, row=2, col=1)
fig.add_annotation(x=ci_upper, y=1.1, text=f"{ci_upper:.2f}", 
                  showarrow=False, row=2, col=1)

# Interpretation
interpretation_text = [
    f"We are 95% confident that the true average sleep time",
    f"for UCSB students is between {ci_lower:.2f} and {ci_upper:.2f} hours.",
    f"",
    f"This means if we repeated this study 100 times,",
    f"about 95 of our intervals would contain the true mean."
]

fig.add_trace(
    go.Scatter(x=[0.5], y=[0.5], mode='text',
              text="<br>".join(interpretation_text),
              textfont=dict(size=14),
              showlegend=False),
    row=2, col=2
)

fig.update_layout(height=700, showlegend=False,
                 title="Complete Analysis: UCSB Student Sleep Study")

# Update axes
fig.update_xaxes(title_text="Hours of Sleep", row=1, col=1)
fig.update_yaxes(title_text="Frequency", row=1, col=1)
fig.update_xaxes(title_text="Hours of Sleep", range=[6.5, 8], row=2, col=1)
fig.update_yaxes(title_text="", showticklabels=False, range=[0.8, 1.2], row=2, col=1)
fig.update_xaxes(showticklabels=False, row=2, col=2)
fig.update_yaxes(showticklabels=False, row=2, col=2)

fig.show()

:::
::: {.column width ="40%"}
**Bottom Line:** We're 95% confident that UCSB students sleep between 6.73 and 7.47 hours per night on average.
:::
::::

---

## Common Mistakes to Avoid ⚠️ {.smaller}

:::: {.columns}
::: {.column width="40%"}
### ❌ Wrong Interpretations

**"95% of students sleep in this range"**
- NO! This is about the population mean, not individual students

**"There's a 95% chance μ is in our interval"**
- NO! $\mu$ is fixed; our interval varies

**"We can be 95% certain"**
- NO! Use "confident" not "certain"
:::
::: {.column}
### ✅ Correct Approach

**"We are 95% confident the population mean is in this interval"**

**Key Reminders:**

- Check conditions before using formulas
  
- Use t-distribution when $\sigma$ is unknown
  
- Larger samples give narrower intervals
  
- Higher confidence gives wider intervals
:::
::::



# Practice Problems 📝 

---

## Problem 1: Coffee Shop Revenue {.scrollable}

A coffee shop owner samples 36 days and finds average daily revenue of $850 with standard deviation $120.

**Your turn:** Calculate a 90% confidence interval for the true average daily revenue.

::: {#my-solution .collapsible-solution}
<button class="solution-toggle">
  Show Solution
</button>
<div class="solution-content">

**Given (from the prompt)**  
$n = 36,\; \bar{x} = \$850,\; s = \$120,\; \text{confidence level} = 90\%$

---

### Step 1 – Conditions  
* $n \ge 30$  ⇒ a $t$‑interval is justified by the Central Limit Theorem.  
* Assume daily revenues are independent.

### Step 2 – Critical value  
$\alpha = 1-0.90 = 0.10 \;\Rightarrow\; \alpha/2 = 0.05$  
Degrees of freedom: $df = n-1 = 35$  
$\displaystyle t^{\star}_{0.90,\,35} \approx 1.690$

### Step 3 – Standard error  
$$SE = \frac{s}{\sqrt{n}}
        = \frac{120}{\sqrt{36}}
        = \frac{120}{6}
        = \$20$$

### Step 4 – Margin of error  
$$ME = t^{\star}\; SE
       = 1.690 \times \$20
       = \$33.8$$

### Step 5 – Confidence interval  
$$\bar{x} \pm ME
     = 850 \pm 33.8
     \;\Longrightarrow\;
     (\$816.2,\; \$883.8)$$

**Interpretation** – We are **90 % confident** that the true mean daily revenue lies between \$816.20 and \$883.80.
</div>
:::

---

## Problem 2: Student Survey {.scrollable}

In a survey of 400 students, 280 say they would recommend their major to a friend.

**Your turn:** 

1. Calculate the sample proportion
   
2. Build a $95\%$ confidence interval  
   
3. Check if conditions are met

::: {#my-solution .collapsible-solution}
<button class="solution-toggle">
  Show Solution
</button>
<div class="solution-content">


**Given (from the survey)**  
$n = 400,\; x = 280$ “yes” responses

---

### Step 1 – Sample proportion  
$$\hat{p} = \frac{x}{n} = \frac{280}{400} = 0.70$$

### Step 2 – Conditions for a $z$‑interval  
$n\hat{p} = 400(0.70)=280 \ge 10$  
$n(1-\hat{p}) = 400(0.30)=120 \ge 10$  
Both counts ≥ 10, so the normal approximation is appropriate.

### Step 3 – Standard error  
$$SE = \sqrt{\frac{\hat{p}(1-\hat{p})}{n}}
        = \sqrt{\frac{0.70(0.30)}{400}}
        = \sqrt{0.000525}
        \approx 0.0229$$

### Step 4 – Critical value & margin of error  
For 95 % confidence, $z^{\star} = 1.96$  

$$ME = z^{\star}\; SE
       = 1.96 \times 0.0229
       \approx 0.045$$

### Step 5 – Confidence interval  
$$\hat{p} \pm ME
     = 0.70 \pm 0.045
     \;\Longrightarrow\;
     (0.655,\; 0.745)$$

**Interpretation** – We are **95 % confident** that between **65.5 % and 74.5 %** of all students would recommend their major to a friend.
</div>
:::



## Looking Ahead: Hypothesis Testing 🔮 

:::: {.columns}
::: {.column}
**Next week we'll learn:**

- How to test specific claims about populations
  
- When to reject or fail to reject hypotheses  
  
- The connection between confidence intervals and hypothesis tests

:::

::: {.column}

In [ ]:
#| eval: false

# Preview of hypothesis testing
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=("Confidence Intervals", "Hypothesis Testing")
)

# CI approach
x = np.linspace(45, 55, 1000)
y = norm.pdf(x, 50, 2)

fig.add_trace(
    go.Scatter(x=x, y=y, mode='lines', name='Sampling Distribution',
              line=dict(color=colors['primary'], width=3)),
    row=1, col=1
)

# Add CI
ci_x = x[(x >= 46.08) & (x <= 53.92)]
ci_y = y[(x >= 46.08) & (x <= 53.92)]
fig.add_trace(
    go.Scatter(x=ci_x, y=ci_y, fill='tonexty', mode='none',
              fillcolor=f'rgba(59, 130, 246, 0.3)', name='95% CI'),
    row=1, col=1
)

# Hypothesis testing approach
fig.add_trace(
    go.Scatter(x=x, y=y, mode='lines', name='Sampling Distribution',
              line=dict(color=colors['danger'], width=3)),
    row=1, col=2
)

# Add rejection regions
reject_left = x[x <= 46.08]
reject_right = x[x >= 53.92]
fig.add_trace(
    go.Scatter(x=reject_left, y=norm.pdf(reject_left, 50, 2), 
              fill='tonexty', mode='none',
              fillcolor=f'rgba(239, 68, 68, 0.3)', name='Reject H₀'),
    row=1, col=2
)
fig.add_trace(
    go.Scatter(x=reject_right, y=norm.pdf(reject_right, 50, 2), 
              fill='tonexty', mode='none',
              fillcolor=f'rgba(239, 68, 68, 0.3)', name='Reject H₀'),
    row=1, col=2
)

fig.update_layout(height=400, width=600, showlegend=False)
fig.show()

:::
::: {.column}

:::
::::



## Key Takeaways 🎯 {.smaller}

:::: {.columns}
::: {.column}
**Big Ideas:**

1. **Samples vary** - confidence intervals capture this uncertainty
   
2. **Larger samples** give more precise estimates  
   
3. **Higher confidence** means wider intervals
   
4. **The CLT** makes normal-based inference possible
:::
::: {.column}
**Practical Skills:**

- Build CIs for means and proportions
  
- Interpret confidence correctly
  
- Plan sample sizes for desired precision
  
- Avoid common interpretation mistakes
:::
::::


## Comprehensive Resources 📚

:::: {.columns}
::: {.column}
### 📖 Required Reading
- **OpenIntro Statistics**
  - Section 1.3: Sampling principles and strategies
  - Section 3.3: Confidence intervals for a mean
  - Section 4.1: Central Limit Theorem
  - Section 7.1.1 The distribution of $\bar x$
  - Section7.1.2 Evaluating the two conditions required for modeling $\bar x$
  - Section 7.1.3 Introducing the $t$-distribution 


### 🎥 Video Resources
- [**Khan Academy:** Central Limit Theorem](https://www.khanacademy.org/math/ap-statistics/sampling-distribution-ap/what-is-sampling-distribution/v/central-limit-theorem)
- [**StatQuest:** Confidence Intervals Explained](https://www.youtube.com/watch?v=TqOeMYtOc1w) 
- [**3Blue1Brown:** Central Limit Theorem Visualization](https://www.3blue1brown.com/lessons/clt)


:::

::: {.column}

### 💻 Interactive Tools
- [**Seeing Theory:** Probability Visualizations](https://seeing-theory.brown.edu/)
- [**Rossman & Chance Applets:** Sampling Distributions](https://www.rossmanchance.com/applets/OneSample53.html?population=model)
- [**Central Limit Theorem Simulator**](http://www.ltcconline.net/greenl/java/Statistics/clt/cltsimulation.html)

:::
::::

## Questions? 🤔 

:::: {.columns}
::: {.column}
**Office Hours:** Thursday 11AM (link on Canvas)  
**Email:** nmathlouthi@ucsb.edu  


*"The goal is not to eliminate uncertainty, but to understand and work with it"*
:::
::: {.column}

:::
::::





<script>
// Direct solution toggle implementation
console.log('Solution toggle script loaded');

// Function to setup toggles
function setupToggles() {
  console.log('Setting up toggles...');
  const buttons = document.querySelectorAll('.solution-toggle');
  console.log('Found buttons:', buttons.length);
  
  buttons.forEach((btn, i) => {
    console.log('Setting up button', i + 1);
    
    // Clear any existing handlers
    btn.onclick = null;
    
    // Add click handler
    btn.onclick = function(e) {
      e.preventDefault();
      e.stopPropagation();
      console.log('Button clicked!');
      
      const parent = this.closest('.collapsible-solution');
      if (parent) {
        parent.classList.toggle('open');
        this.textContent = parent.classList.contains('open') ? 'Hide Solution' : 'Show Solution';
        console.log('Toggled! Open:', parent.classList.contains('open'));
      }
    };
  });
}

// Try multiple times
document.addEventListener('DOMContentLoaded', setupToggles);
window.addEventListener('load', setupToggles);
setTimeout(setupToggles, 500);
setTimeout(setupToggles, 1000);
setTimeout(setupToggles, 2000);

// For Reveal.js
if (typeof Reveal !== 'undefined') {
  Reveal.addEventListener('ready', setupToggles);
}
</script>